In [3]:
import requests
from urllib import parse
from bs4 import BeautifulSoup
import pymongo
import time
import random

In [4]:
# mongoDB 연결 and crawling_review collection 생성
conn = pymongo.MongoClient()
db = conn.crawling_review